# Initialize Data

In [ ]:
import pandas as pd
from evaluation import evaluate_metrics

df = pd.read_parquet("data/test_data.parquet")
df["passage_text"] = df.apply(lambda x: x["passages"]["passage_text"], axis=1)
df["is_selected"] = df.apply(lambda x: x["passages"]["is_selected"], axis=1)
df.drop("passages", axis=1, inplace=True)
df.set_index("query_id", inplace=True)
df.head()

,answers,query,query_type,wellFormedAnswers,passage_text,is_selected
query_id,,,,,,
0,[Yes],does human hair stop squirrels,description,[],[We have been feeding our back yard squirrels ...,"[0, 0, 1, 0, 0, 0, 0]"
1,[Fossil fuels are basically the remains of ani...,what are the benefits of fossil fuels,description,[],[The biggest advantage of using fossil fuels i...,"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
2,[The apothem of a regular polygon is a line se...,what is a apothem,description,[],[Apothem. The apothem of a regular polygon is ...,"[0, 0, 0, 0, 0, 1, 0, 0, 0]"
3,[$45 to $210. 2],average cost for custom canopy,numeric,[],"[Congratulations! You have found BuyShade.com,...","[0, 0, 0, 0, 0, 1, 0, 0, 0]"
4,[It is the collection of physical elements tha...,what is a hardware in a computer,description,[],"[Hardware is best described as a device, such ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"


In [4]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Rank using OpenAI

In [ ]:
from ranking.rank_openai import rank_passages_openai

df["ranked_passages"] = df.parallel_apply(lambda x: rank_passages_openai(x["query"], x["passage_text"]), axis=1)

In [6]:
result_openai = evaluate_metrics(df, relevance_col='is_selected', rank_col='ranked_passages')
result_openai

{'MRR': 0.5346607862488691,
 'nDCG': 0.637723390246129,
 'Precision at 3': 0.2360621761658031,
 'Recall at 3': 0.6452055267702936,
 'MAP': 0.5275370719905694}

## Rank using TF-IDF

In [7]:
from ranking.rank_tfidf import rank_passages_tfidf

df["ranked_passages_tfidf"] = df.parallel_apply(lambda x: rank_passages_tfidf(x["query"], x["passage_text"]), axis=1)

In [8]:
result_tfidf = evaluate_metrics(df, relevance_col='is_selected', rank_col='ranked_passages_tfidf')
result_tfidf

{'MRR': 0.451944896784275,
 'nDCG': 0.5737061735586872,
 'Precision at 3': 0.19582037996545767,
 'Recall at 3': 0.5317081174438688,
 'MAP': 0.4455216499438003}